In [ ]:
pip install tf-nightly

In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.rcParams["figure.figsize"] = (20,12)
figure(figsize=(100, 80), dpi=80)

<Figure size 8000x6400 with 0 Axes>

<Figure size 8000x6400 with 0 Axes>

In [ ]:
tf.random.set_seed(12345)

In [ ]:
df = pd.read_csv('alicante_Autobuses_Flow.csv')

In [ ]:
df

In [ ]:
df = df.drop_duplicates(subset = 'time', keep='first')

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
Time = []

for i in range(0, len(df)):
    Time.append(datetime.datetime.fromtimestamp(df['time'][i]).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
df['time'] = Time

In [ ]:
df['value'].isnull().sum()

In [ ]:
df.columns = ['Time','Values']

In [ ]:
df

In [ ]:
df['Time'] = pd.to_datetime(df['Time'])
df1 = df.set_index('Time')
Min_summary = df1.resample('30T').mean()

In [ ]:
Min_summary

In [ ]:
index = np.arange(len(Min_summary))

In [ ]:
df = pd.DataFrame({
    'Time':Min_summary.index,
    'Values':Min_summary['Values']
})

In [ ]:
df.index = index

In [ ]:
df

In [ ]:
df.to_csv('data_autobus.csv')

In [ ]:
df['Values'].isnull().sum()

In [ ]:
Values = df['Values'].values

In [ ]:
Values.shape

In [ ]:
import numpy.ma as ma

In [ ]:
X = ma.masked_invalid(Values).reshape(-1,1)
X[X==0]=ma.masked

In [ ]:
X.min()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Function for Scaler Application
def Scaler(X):
    #scaler = MinMaxScaler()
    # Fit Scaler
    #scaler_X = scaler.fit(X)
    # Transform Data
    #X_ = scaler_X.transform(X)
    minX = X.min()
    X+=minX
    maxX = X.max()
    X = X/maxX
    return minX,maxX,X

In [ ]:
minX,maxX,scaled_x = Scaler(ma.compress_rows(X))

In [ ]:
scaled_x.max()

In [ ]:
X[-100:]

In [ ]:
scaled_x.min()

In [ ]:
def inverse_scaler(scaled_x,minX,maxX):
    return scaled_x*maxX-minX

In [ ]:
plt.plot(inverse_scaler(scaled_x, minX, maxX))
plt.xlabel('Time Delays (K)', fontsize = 20)
plt.ylabel('Flow (L/h)', fontsize = 20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig('data.jpg')
plt.show()

In [ ]:
plt.plot(inverse_scaler(scaled_x[:336], minX, maxX), label = 'Sample of Data')
plt.legend(loc='best', fontsize=20)
plt.xlabel('Time Delays (K)', fontsize = 30)
plt.ylabel('Flow (L/h)', fontsize = 20)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.savefig('7DaysSample.jpg')
plt.show()

In [ ]:
scaled_x*maxX-minX

In [ ]:
scaled_X = ma.zeros(X.shape)
scaled_X[~X.mask[:,0]] = scaled_x
scaled_X[X.mask] = ma.masked


In [ ]:
import h5py

In [ ]:
f = h5py.File("autobuses_processed_data.h5","w")
v_ = f.create_dataset("raw_values",shape=Values.shape,dtype="float")
v_[...] = Values

x_ = f.create_dataset("scaled_x",shape=scaled_X.shape,dtype="float")
x_[...] = scaled_X

minx_ = f.create_dataset("minX",shape=(1,),dtype="float")
minx_[...] = minX

maxx_ = f.create_dataset("maxX",shape=(1,),dtype="float")
maxx_[...] = maxX


x_mask = f.create_dataset("x_mask",shape=scaled_X.shape,dtype="Bool")
x_mask[...] = scaled_X.mask

f.close()

In [ ]:
f = h5py.File("autobuses_processed_data.h5","r")
scaled_X_ = ma.array(f["scaled_x"])
scaled_X_.mask = ma.array(f["x_mask"])
f.close()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(scaled_X_[:500])
plt.plot(scaled_X[:500])
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()